検索システムを構築することを考えたとき，ある検索語に対して，複数の文書がヒットすることがある．

このような状況ではランキングが必要となる．ランキングは検索語を文書と見なして，文書と文書の類似度から求める．



# 文書のランキング


## 特徴語集合の類似性

特徴語の類似性から文書のランキングを求める．
一般には次の三つがある. 

- Jaccard
- Dice
- Simposon


In [25]:
import collections
import importlib
from  janome.tokenizer import Tokenizer
import re
import pathlib
from py import my_module as mm
from pprint import pprint
from gensim import corpora, models
importlib.reload(mm)

from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import ExtractAttributeFilter
from janome.tokenfilter import POSStopFilter
from janome.tokenfilter import POSKeepFilter

In [14]:
data_fmt = "irpb-files/data/ch04/{:d}.txt"
book_texts = [mm.get_string_from_file(data_fmt.format(i)) for i in range(10)]

tfidf_model, dic, book_weights = mm.get_tfidfmodel_and_weights(book_texts)

keyword_lists = [[x0 for x0, *_ in w[:10]] for w in book_weights]

results = [(x, mm.jaccard(keyword_lists[9], keyword_lists[x])) for x in range(9)]

results.sort(key = lambda x: x[1], reverse = True)

with open("irpb-files/data/ch04/book-titles.txt", encoding = "UTF-8") as f:
    titles = f.read().strip().split("\n")



In [15]:
for x in range(9):
    print("%s %.4f" % (titles[results[x][0]], results[x][1]))

ブラックホールと時空の方程式 0.1111
逆数学 0.1111
64の事例からわかる金属腐食の対策 0.0526
Coq/SSReflect/MathCompによる定理証明 0.0526
基礎からわかる高分子材料 0.0000
ゼロからはじめるVisual_C#入門 0.0000
実践_地域・まちづくりワーク 0.0000
応用数学問題集 0.0000
生態系生態学(第2版) 0.0000


## ベクトル空間モデル

TF・IDFは特徴語としての性能を示すものであった．

ここではTF・IDFを語が文書を特徴付に寄与してい度合いと考え，その分布が文書の内容を表していると考える．

これは文章を特徴語のTF・IDFで示されるベクトルだと考えることになる．これをベクトル空間モデルという. 

In [16]:
from gensim import corpora, models

texts = ["花より団子．とにかく団子", "みたらしよりあんこ"]

words = [mm.get_words(text, keep_pos = ["名詞"]) for text in texts]
dic = corpora.Dictionary(words)

for i in range(len(dic)):
    print("dic[%d] = %s" % (i, dic[i]))


dic[0] = 団子
dic[1] = 花
dic[2] = あんこ
dic[3] = みたらし


In [24]:
bows = [dic.doc2bow(w) for w in words]
tfidf = models.TfidfModel(bows)
weights = tfidf[bows[0]]
weights = [(i, round(j, 4)) for i, j in weights]
print("wegihts = ", weights)


wegihts =  [(0, 0.8944), (1, 0.4472)]


wegihts =  [(0, 2), (1, 1)]


In [31]:
data_fmt = "irpb-files/data/ch04/{:d}.txt"
book_texts = [mm.get_string_from_file(data_fmt.format(i)) for i in range(10)]

titles = mm.get_list_from_file("irpb-files/data/ch04/book-titles.txt")

result = mm.vsm_search(book_texts[:-1], book_texts[9])


for x in range(9):
    print("dic[%s] = %s" % (titles[result[x][0]], result[x][1]))


dic[逆数学] = 0.420418
dic[ブラックホールと時空の方程式] = 0.27373135
dic[Coq/SSReflect/MathCompによる定理証明] = 0.17463519
dic[応用数学問題集] = 0.11947917
dic[64の事例からわかる金属腐食の対策] = 0.06601203
dic[実践_地域・まちづくりワーク] = 0.038086623
dic[ゼロからはじめるVisual_C#入門] = 0.02615247
dic[生態系生態学(第2版)] = 0.025868826
dic[基礎からわかる高分子材料] = 0.024284441
